# Wrangle Exercises

## Data Acquisition

This exercises uses the `case.csv`, `dept.csv`, and `source.csv` files from the san antonio 311 call dataset.

## 1. Read the case, department, and source data into their own spark dataframes.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [2]:
df_source = spark.read.csv("source.csv", sep = ',', header=True, inferSchema=True)
df_source.show(3)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
+---------+----------------+
only showing top 3 rows



In [3]:
df = spark.read.csv("case.csv", sep = ',', header=True, inferSchema=True)
df.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [4]:
dept = spark.read.csv("dept.csv", sep = ',', header=True, inferSchema=True)
dept.show(1, vertical=True)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row



## 2. Let's see how writing to the local disk works in spark:

* Write the code necessary to store the source data in both csv and json format, store these as `sources_csv` and `sources_json`
* Inspect your folder structure. What do you notice?

In [5]:
df_source.write.json("souces_json", mode="overwrite")

## 3. Inspect the data in your dataframes. 

* Are the data types appropriate?
* Write the code necessary to cast the values to the appropriate types.

In [6]:
# Turn values to bools
df = df.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)

In [7]:
# Turn district to string
df = df.withColumn("council_district", col("council_district").cast("string"))

In [8]:
# Edit format of the address
df = df.withColumn("request_address", trim(lower(df.request_address)))

In [9]:
# turn days late to weeks 
df = df.withColumn(
    "num_weeks_late", expr("num_days_late / 7 AS num_weeks_late")
)

In [10]:
# change dates to timestamp
fmt = "M/d/yy H:mm"
df = (
    df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_closed_date", fmt))
    .withColumn("SLA_due_date", to_timestamp("SLA_due_date", fmt))
)

In [11]:
df.select(max(df.case_closed_date)).show()

+---------------------+
|max(case_closed_date)|
+---------------------+
|  2018-08-08 10:38:00|
+---------------------+



In [12]:
df = (
    df.withColumn(
        "case_age", datediff(to_date(lit("2018-08-08")), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("case_closed")).show(5)

df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("! case_closed")).show(5)

+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|   case_closed_date|case_age|days_to_closed|case_lifetime|
+-----------+-------------------+-------------------+--------+--------------+-------------+
|       true|2018-01-01 00:42:00|2018-01-01 12:29:00|     219|             0|            0|
|       true|2018-01-01 00:46:00|2018-01-03 08:11:00|     219|             2|            2|
|       true|2018-01-01 00:48:00|2018-01-02 07:57:00|     219|             1|            1|
|       true|2018-01-01 01:29:00|2018-01-02 08:13:00|     219|             1|            1|
|       true|2018-01-01 01:34:00|2018-01-01 13:29:00|     219|             0|            0|
+-----------+-------------------+-------------------+--------+--------------+-------------+
only showing top 5 rows

+-----------+-------------------+----------------+--------+--------------+-------------+
|case_closed|   case_opened_date|case_closed_date|case_age

In [13]:
df = (
    df
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

-------

# Prepare Exercises

## 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [14]:
df.show(1, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 case_age             | 219                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
only showing top 1 row



In [15]:
(
    df.select(df.SLA_due_date, df.case_opened_date, df.case_closed_date, df.case_closed)
    .filter(df.case_closed == "false")
    .sort("case_opened_date")
).first()

Row(SLA_due_date=datetime.datetime(2017, 1, 17, 8, 30), case_opened_date=datetime.datetime(2017, 1, 1, 13, 48), case_closed_date=None, case_closed=False)

## 2. How many Stray Animal cases are there?

In [16]:
df.filter(df.service_request_type == "Stray Animal").count()

26760

## 3. How many service requests that are assigned to the Field Operations department (`dept_division`) are not classified as "Officer Standby" request type (`service_request_type`)?

In [17]:
(
    df.filter(df.dept_division == "Field Operations")
    .filter(df.service_request_type != "Officer Standby")
).count()

113902

## 4. Convert the council_district column to a string column.

In [18]:
# Done in the acquire section

## 5. Extract the year from the case_closed_date column.

In [19]:
(
    df.withColumn("year", year("case_closed_date"))
    .select("year")
    .show()
)

+----+
|year|
+----+
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
+----+
only showing top 20 rows



## 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [20]:
# turn days late to weeks 
df = df.withColumn(
    "num_hours_late", expr("num_days_late * 24 AS num_hours_late")
)

In [21]:
df.show(1, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 case_age             | 219                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 num_hours_late       | -23964.210

## 7. Join the case data with the source and department data.

In [22]:
df_source.show(1)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
+---------+----------------+
only showing top 1 row



In [23]:
df.select("source_id").show(100)

+---------+
|source_id|
+---------+
| svcCRMLS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
|   139344|
|   139344|
|  ns16326|
| svcCRMSS|
| svcCRMSS|
| svcCRMSS|
|   139344|
|   139344|
|   139344|
|  ns16326|
|   139344|
|  ns16326|
|  ns16326|
|  ns16326|
|  ns16326|
|  ns16326|
|   139344|
|   139344|
|  ns16326|
|   139344|
|  ns16326|
|   139344|
|  ns16326|
|   139344|
|  ns16326|
|  ns16326|
|  ns16326|
|   139344|
|  ns16326|
|   139344|
|  ns16326|
| svcCRMSS|
|   139344|
|   139344|
|  n

In [24]:
df = df.join(df_source, "source_id", "left")

In [25]:
df.show(50, vertical = True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 case_age             | 219                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 num_hours_late       | -23964.210

## 8. Are there any cases that do not have a request source?

In [26]:
df.filter(col("source_id").isNull()).show(vertical=True)

(0 rows)



In [27]:
df.filter(col("source_username").isNull()).show(vertical=True)

# We don't seem to have any null values - meaning that there is always
# a source_id listed

(0 rows)



## 9 What are the top 10 service request types in terms of number of requests?

In [28]:
(
    df.groupBy(df.service_request_type)
    .agg(count(df.case_id).alias("count"))
    .sort(col("count").desc())
    .show(10, truncate= False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



## 10. What are the top 10 service request types in terms of average days late?

In [40]:
(
    df.where("case_late")
    .groupby(df.service_request_type)
    .agg(round(mean(df.num_days_late), 0).alias("mean_days"), count("*").alias("n_cases"))
    .sort(col("mean_days").desc())
    .show(10, truncate = False)
)

+--------------------------------------+---------+-------+
|service_request_type                  |mean_days|n_cases|
+--------------------------------------+---------+-------+
|Zoning: Recycle Yard                  |211.0    |132    |
|Zoning: Junk Yards                    |200.0    |262    |
|Structure/Housing Maintenance         |190.0    |51     |
|Donation Container Enforcement        |171.0    |122    |
|Storage of Used Mattress              |164.0    |7      |
|Labeling for Used Mattress            |162.0    |7      |
|Record Keeping of Used Mattresses     |154.0    |7      |
|Signage Requied for Sale of Used Mattr|152.0    |12     |
|Traffic Signal Graffiti               |138.0    |4      |
|License Requied Used Mattress Sales   |129.0    |7      |
+--------------------------------------+---------+-------+
only showing top 10 rows



## 11. Does number of days late depend on department?

In [47]:
(
    df.filter("case_late")
    .groupby(df.department)
    .agg(round(mean(df.num_days_late), 1).alias("avg_days_late"), count("num_days_late").alias("n_cases_late"))
    .sort(col('avg_days_late').desc())
    .show(10, truncate = False)
)

# It seems tha customer service has an extremly understaff team,
# since they are the only ones that actually seem to be having 
# a lot of days late

+------------------------+-------------+------------+
|department              |avg_days_late|n_cases_late|
+------------------------+-------------+------------+
|Customer Service        |88.2         |2035        |
|DSD/Code Enforcement    |49.5         |26439       |
|Animal Care Services    |23.4         |23751       |
|Parks & Recreation      |22.4         |3810        |
|Trans & Cap Improvements|10.7         |5529        |
|Solid Waste             |7.1          |33729       |
|Metro Health            |6.5          |854         |
+------------------------+-------------+------------+



## 12. How do number of days late depend on department and request type?

In [48]:
(
    df.filter("case_late")
    .groupby(df.department, df.service_request_type)
    .agg(round(mean(df.num_days_late),0).alias("avg_days_late"))
    .sort(col("avg_days_late").desc())
    .show(10, truncate = False)
)



+------------------------+--------------------------------------+-------------+
|department              |service_request_type                  |avg_days_late|
+------------------------+--------------------------------------+-------------+
|DSD/Code Enforcement    |Zoning: Recycle Yard                  |211.0        |
|DSD/Code Enforcement    |Zoning: Junk Yards                    |200.0        |
|DSD/Code Enforcement    |Structure/Housing Maintenance         |190.0        |
|DSD/Code Enforcement    |Donation Container Enforcement        |171.0        |
|DSD/Code Enforcement    |Storage of Used Mattress              |164.0        |
|DSD/Code Enforcement    |Labeling for Used Mattress            |162.0        |
|DSD/Code Enforcement    |Record Keeping of Used Mattresses     |154.0        |
|DSD/Code Enforcement    |Signage Requied for Sale of Used Mattr|152.0        |
|Trans & Cap Improvements|Traffic Signal Graffiti               |138.0        |
|DSD/Code Enforcement    |License Requie

---------

## Test that py file works

In [32]:
import wrangle

In [35]:
df_new = wrangle.wrangle_data()

In [36]:
df_new.show(1, vertical=True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 SLA_due_date         | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 case_age             | 219                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 source_username      | svcCRMLS  